# Get alert history for all devices

In [1]:
import os
import sys
from mxnet.gluon import data as gdata

def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'fashion-mnist')):
    """Download the fashion mnist dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.FashionMNIST(root=root, train=True)
    mnist_test = gdata.vision.FashionMNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4
    
    print(mnist_train.transform_first(transformer))
    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    
    print(mnist_test.transform_first(transformer))
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter


In [2]:
import gluonbook as gb
from mxnet import autograd, nd

num_inputs = 784
num_outputs = 10
batch_size = 256

train_iter, test_iter = load_data_fashion_mnist(batch_size)

W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
b = nd.zeros(num_outputs)
params = [W, b]

for param in params:
    param.attach_grad()

In [3]:
def softmax(X):
    X_exp = nd.exp(X)
    partition = X_exp.sum(axis=1, keepdims=True)
    return X_exp / partition

In [4]:
def net(X):
    return softmax(nd.dot(X.reshape(-1, num_inputs), W) + b)

In [5]:
def cross_entropy(y_hat, y):
    return - nd.pick(y_hat, y).log()

In [6]:
def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1) == y.astype('float32')).mean().asscalar()

In [7]:
def evaluate_accuracy(data_iter, net):
    acc = 0.
    for X, y in data_iter:
        acc += accuracy(net(X), y)
    return acc / len(data_iter)

In [8]:
evaluate_accuracy(test_iter, net)

0.09462890625

In [9]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [10]:
from mxnet import autograd

num_epochs, learning_rate = 5, 0.1

for epoch in range(num_epochs):
    train_loss = 0.
    train_acc = 0.
    for X, y in train_iter:
        with autograd.record():
            y_hat = net(X)
            loss = cross_entropy(y_hat, y)
        loss.backward()
        
        SGD(params, learning_rate / batch_size)
        
        train_loss += loss.mean().asscalar()
        train_acc += accuracy(y_hat, y)
        
    test_acc = evaluate_accuracy(test_iter, net)

    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch + 1, train_loss/len(train_iter), train_acc/len(train_iter), test_acc))


Epoch 1. Loss: 0.806503, Train acc 0.738525, Test acc 0.782813
Epoch 2. Loss: 0.609133, Train acc 0.798742, Test acc 0.819922
Epoch 3. Loss: 0.594843, Train acc 0.802881, Test acc 0.810156
Epoch 4. Loss: 0.593439, Train acc 0.806416, Test acc 0.823145
Epoch 5. Loss: 0.547939, Train acc 0.817453, Test acc 0.827637
